* In this assignment you will be implementing an encoder model that uses just **Masked Language Modelling (MLM)** objective.
* We will use a simple BERT with the following modifications
  * We just use MLM (just masking words) and **skip** NSP (Next Sentence Prediction) objective
  * Therefore, we won't use [CLS] token
* Again, it is absolutely fine if your loss value does not match with the one given here.
* Just ensure that the model overfits the training data
* You may increase the size of the training data if you want to test your implementation. In that case, we recommend you to use the tokenizer library from Hugging face



# Installations

In [1]:
!pip install torchdata==0.6.0 # to be compatible with torch 2.0
!pip install portalocker==2.0.0
!pip install -U torchtext==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Common Imports

In [2]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

#text lib
import torchtext

# tokenizer
from torchtext.data.utils import get_tokenizer

#build vocabulary
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator

# get input_ids (numericalization)
from torchtext.transforms import VocabTransform

# get embeddings
from torch.nn import Embedding

from  pprint import pprint
from yaml import safe_load
import copy
import numpy as np
import math

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenize the given text

In [4]:
batch_size = 10

In [5]:
class Tokenizer(object):

  def __init__(self,text):
    self.text = text
    self.word_tokenizer = get_tokenizer(tokenizer="basic_english",language='en')
    self.vocab_size = None

  def get_tokens(self):
    for sentence in self.text.strip().split('\n'):
      yield self.word_tokenizer(sentence)

  def build_vocab(self):
    v = build_vocab_from_iterator(self.get_tokens(),
                                  min_freq=1,specials=['<unk>','<mask>'])
    v.set_default_index(v['<unk>']) # index of OOV
    self.vocab_size = len(v)
    return v

  def token_ids(self):
    v = self.build_vocab()
    vt = VocabTransform(v)
    num_tokens = len(self.word_tokenizer(self.text))
    max_seq_len = np.ceil(num_tokens/batch_size)
    data = torch.zeros(size=(1,num_tokens))
    data = vt(self.word_tokenizer(self.text))
    data = torch.tensor(data,dtype=torch.int64)
    return data.reshape(batch_size,torch.tensor(max_seq_len,dtype=torch.int64))

In [6]:
text = """Best known for the invention of Error Correcting Codes, he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines.
Reflecting on the significant benefits I received from Hamming, I decided to develop a tribute to his legacy. There has not been a previous biography of Hamming, and the few articles about him restate known facts and assumptions and leave us with open questions.
One thought drove me as I developed this legacy project: An individual's legacy is more than a list of their attempts and accomplishments. Their tribute should also reveal the succeeding generations they inspired and enabled and what each attempted and achieved.
This book is a unique genre containing my version of a biography that intertwines the story "of a life" and a multi-player memoir with particular events and turning points recalled by those, including me, who he inspired and enabled.
Five years of research uncovered the people, places, opportunities, events, and influences that shaped Hamming. I discovered unpublished information, stories, photographs, videos, and personal remembrances to chronicle his life, which helped me put Hamming's
legacy in the context I wanted.The result demonstrates many exceptional qualities, including his noble pursuit of excellence and helping others. Hamming paid attention to the details, his writings continue to influence, and his guidance is a timeless gift to the world.
This biography is part of """

In [7]:
Tk = Tokenizer(text)

In [8]:
input_ids = Tk.token_ids()
print(input_ids.shape)

torch.Size([10, 26])


* We need to mask some words randomly based on the mask probability
* The token id for the [mask] is 1
* The function given below takes in the input ids and replaces some of the ids by 1 (token id for the [mask])
* Since the loss is computed only over the predictions of masked tokens, we replace all non-masked input ids by -100

In [9]:
def getdata(ip_ids, mask_token_id, mask_prob = 0.2):
    masked_ids = copy.deepcopy(ip_ids)
    mask_random_idx = torch.randn_like(ip_ids, dtype = torch.float64)>(1 - mask_prob)
    masked_ids[mask_random_idx] = mask_token_id
    labels = copy.deepcopy(ip_ids)
    neg_mask = ~mask_random_idx
    labels[neg_mask] = torch.tensor(-100)
    return (masked_ids, labels, mask_random_idx)

In [10]:
mask_token_id = torch.tensor([1], dtype = torch.int64)

x, y, mask_mtx = getdata(input_ids, mask_token_id)

print(x[0, :], '\n', y[0,:])

tensor([  1,   1,  69,   5,  85,   1,  63,  53,  49,   2,  20, 148,   6, 139,
        110,  29,   1,   9,   1,   1, 112, 129,   8,   1,  59,   1]) 
 tensor([  45,   23, -100, -100, -100,    7, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100,   36, -100,   89,    3, -100, -100, -100,   96,
        -100,    4])


* Now we have our inputs and labels stored in x and y,respectively
* It is always good to test the implementation by displaying the input sentence with masked tokens

In [11]:
v = Tk.build_vocab()
words = []

for idx in x[0, :]:
  words.append(v.vocab.get_itos()[idx.item()])

print(' '.join(words))

<mask> <mask> for the invention <mask> error correcting codes , he was a true polymath who <mask> his <mask> <mask> problem-solving skills to <mask> disciplines <mask>


* Also display the words that are masked

In [12]:
words = []

for idx in y[0, :]:
  if idx != -100:
    words.append(v.vocab.get_itos()[idx.item()])

print(' '.join(words))

best known of applied mathematical and numerous .


# Configuration

In [13]:
vocab_size = Tk.vocab_size

seq_len = x.shape[1]

embed_dim = 32

dmodel = embed_dim

dq = torch.tensor(4)

dk = torch.tensor(4)

dv = torch.tensor(4)

heads = torch.tensor(8)

d_ff = 4 * dmodel

# Model

In [14]:
class MHA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads):
        super(MHA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(dmodel, heads * dq, seed = 43))
        self.WK = nn.Parameter(self.init_param(dmodel, heads * dk, seed = 44))
        self.WV = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 45))
        self.WO = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 46))

        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, H = None):
        '''
        Input  : Size  [BS, T, dmodel]
        Output : Size  [BS, T, dmodel]
        '''
        BS, T, _ = H.shape

        Q = torch.matmul(H, self.WQ.T)
        K = torch.matmul(H, self.WK.T)
        V = torch.matmul(H, self.WV.T)

        Q = Q.view(BS, T, self.heads, self.dq).transpose(1, 2)
        K = K.view(BS, T, self.heads, self.dk).transpose(1, 2)
        V = V.view(BS, T, self.heads, self.dv).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor
        attention = F.softmax(attention_scores, dim = -1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V)
        out = out.transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out


class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        '''
        Input  : Size [BS, T, dmodel]
        Output : Size [BS, T, dmodel]
        '''
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out


class Prediction(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(Prediction, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        out = self.linear(representations)
        # out = self.dropout(out)
        return out


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


class Embed(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(Embed, self).__init__()
        self.token_embed = nn.Embedding(vocab_size, embed_dim)
        self.pos_embed = PositionalEncoding(embed_dim)

    def forward(self, x):
        token_embedding = self.token_embed(x)
        positional_embedding = self.pos_embed(token_embedding)
        return positional_embedding


class EncoderLayer(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, d_ff, heads):
        super(EncoderLayer, self).__init__()
        self.mha = MHA(dmodel, dq, dk, dv, heads)
        self.ffn = FFN(dmodel, d_ff)
        self.layer_norm_1 = nn.LayerNorm(dmodel)
        self.layer_norm_2 = nn.LayerNorm(dmodel)

    def forward(self, x):
        # Multi-head attention
        attn_output = self.mha(x)
        out1 = self.layer_norm_1(x + attn_output)
        # Feed-forward network
        ffn_output = self.ffn(out1)
        out2 = self.layer_norm_2(out1 + ffn_output)
        return out2

In [15]:
class BERT(nn.Module):

    def __init__(self, vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers = 1):
        super(BERT, self).__init__()
        self.embed_lookup = Embed(vocab_size, dmodel)
        self.enc_layers = nn.ModuleList([EncoderLayer(dmodel, dq, dk, dv, d_ff, heads) for _ in range(num_layers)])
        self.predict = Prediction(dmodel, vocab_size)

    def forward(self, input_ids):
        x = self.embed_lookup(input_ids)
        for enc_layer in self.enc_layers:
            x = enc_layer(x)
        out = self.predict(x)
        return out

In [16]:
model = BERT(vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers = 1)

optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

# Training the model

In [17]:
def train(model, token_ids, labels, criterion, optimizer, epochs = 1000):
    model.train()  # Set the model to training mode
    loss_trace = []

    for epoch in range(epochs):
        optimizer.zero_grad()  # Clear gradients
        outputs = model(token_ids)  # Forward pass

        # Flatten the outputs and labels
        outputs_flat = outputs.view(-1, outputs.size(-1))
        labels_flat = labels.view(-1)

        # Filter out the positions where labels are -100
        # These positions do not contribute to the loss
        mask = labels_flat != -100
        outputs_flat = outputs_flat[mask]
        labels_flat = labels_flat[mask]

        # Now the outputs and labels should have the same batch size
        loss = criterion(outputs_flat, labels_flat)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        loss_trace.append(loss.item())

        if epoch % 100 == 0:  # Print loss every 100 epochs
            print(f'Epoch {epoch} Loss: {loss.item()}')

In [18]:
model = BERT(vocab_size, dmodel, dq, dk, dv, d_ff, heads, num_layers = 1)

optimizer = optim.SGD(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss(ignore_index = -100)  # Ignore -100 labels in loss computation

In [19]:
train(model, x, y, criterion, optimizer, epochs = 20000)

Epoch 0 Loss: 5.024461269378662
Epoch 100 Loss: 4.272457599639893
Epoch 200 Loss: 3.7562460899353027
Epoch 300 Loss: 3.3201756477355957
Epoch 400 Loss: 2.8637218475341797
Epoch 500 Loss: 2.3890035152435303
Epoch 600 Loss: 1.9391937255859375
Epoch 700 Loss: 1.5376886129379272
Epoch 800 Loss: 1.1994483470916748
Epoch 900 Loss: 0.9303851127624512
Epoch 1000 Loss: 0.7261582612991333
Epoch 1100 Loss: 0.5742906928062439
Epoch 1200 Loss: 0.4608652591705322
Epoch 1300 Loss: 0.3758087754249573
Epoch 1400 Loss: 0.3108769357204437
Epoch 1500 Loss: 0.26033830642700195
Epoch 1600 Loss: 0.22075656056404114
Epoch 1700 Loss: 0.18954791128635406
Epoch 1800 Loss: 0.164807990193367
Epoch 1900 Loss: 0.14484216272830963
Epoch 2000 Loss: 0.1272018849849701
Epoch 2100 Loss: 0.1067371740937233
Epoch 2200 Loss: 0.09182705730199814
Epoch 2300 Loss: 0.08094567805528641
Epoch 2400 Loss: 0.07237010449171066
Epoch 2500 Loss: 0.06536819040775299
Epoch 2600 Loss: 0.05950483679771423
Epoch 2700 Loss: 0.054493956267833

* The loss will come around 0.02 after 20000 epochs (again, it is absolutely fine if you get a different value)
* Let us predict the masked tokens for all the samples in the tiny dataset

In [20]:
with torch.inference_mode():
  predictions = torch.argmax(model(x), dim = -1)

In [27]:
v = Tk.build_vocab()
masked_words = []
predicted_words=[]

for index,idx in enumerate(y.flatten()):
  # to display only the masked tokens
  if idx != -100:
    masked_words.append(v.vocab.get_itos()[idx.item()])
    predicted_words.append(v.vocab.get_itos()[predictions.flatten()[index].item()])

print('Masked Words -> ')
print(' '.join(masked_words))
print('\n')
print('Predicted Words -> ')
print(' '.join(predicted_words))

Masked Words -> 
best known of applied mathematical and numerous . on i i to develop tribute to . there previous of , few articles restate known and assumptions us with an than their . their reveal the succeeding generations inspired and and each and achieved . is intertwines recalled he inspired and of research the i videos , which put in the and helping a the


Predicted Words -> 
best known of applied mathematical and numerous . on i i to develop tribute to . there previous of , few articles restate known and assumptions us with an than their . their reveal the succeeding generations inspired and and each and achieved . is intertwines recalled he inspired and of research the i videos , which put in the and helping a the
